In [11]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle

In [12]:
page = requests.get('https://maldita.es/malditobulo/')

In [13]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [14]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [15]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [16]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url}
    
    return article

In [17]:
lista_enlaces = []
suficiente = True
n_pag = 1
n_noticias = 1#Las noticias van en bloques de 24. La carga computacional aumentaria mucho si quisiera que fuera mas preciso
                #Pero para este caso de uso y estudio no es necesario.

#Cargo las URLS.
print('Cargando URLs...')

while (len(lista_enlaces) <= n_noticias-1 and suficiente):
    link =  "https://maldita.es/malditobulo/page/" + str(n_pag) + "/"
    arrayurls= get_urls(link, lista_enlaces)
    n_pag = n_pag + 1

    if (n_pag >=10):
        suficiente=False
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/05/28/frap-padre-pablo-iglesias/', 'https://maldita.es/malditobulo/2020/05/28/pajaro-zunzuncito-mas-pequeno-mosca/', 'https://maldita.es/malditobulo/2020/05/28/estudio-tocar-tetas-reduce-ansiedad-hombres-90/', 'https://maldita.es/malditobulo/2020/05/28/podemos-propuesta-prohibir-banderas-espana-balcones/', 'https://maldita.es/malditaciencia/2020/05/28/cadena-italia-cura-coronavirus-autopsias-prohibidas/', 'https://maldita.es/malditobulo/2020/05/27/no-este-documento-del-ministerio-de-sanidad-no-obligaba-a-mantener-a-los-ancianos-con-sintomatologia-de-covid-19-en-residencias-y-a-no-trasladarlos-a-hospitales/', 'https://maldita.es/malditobulo/2020/05/27/coronel-diego-perez-de-los-cobos-voluntario-golpe-23-f-militante-partido-popular/', 'https://maldita.es/malditobulo/2020/05/27/imagen-musulmanes-moros-alimentos-mercedes-alemania/', 'https://maldita.es/malditobulo/2020/05/27/obispo-barquisimeto-venezuela-detenido/', 'https://maldita

In [18]:
lista_articulos

[{'Title': '¿Qué sabemos sobre la vinculación del padre de Pablo Iglesias con el FRAP?',
  'Publication_date': '28/05/2020',
  'Content': 'La portavoz del Partido Popular en el Congreso de los Diputados, Cayetana Álvarez de Toledo, ha asegurado en el Pleno de este miércoles, 27 de mayo, que el vicepresidente segundo del Gobierno, Pablo Iglesias, era “hijo de un terrorista”. Se trata de una acusación que lleva años circulando y que también fue difundida por el actual eurodiputado de Vox Hermann Tertsch, quien fue más allá y aseguró en 2018 que Francisco Javier Iglesias fue detenido por el asesinato del subinspector de Policía José Antonio Fernández Gutiérrez, ocurrido en 1973, y que fue atribuido al Frente Revolucionario Antifascista y Patriota (FRAP).Pablo Iglesias reconoció ser “hijo de un militante del FRAP” en una columna publicada en Público en el año 2012, pero su padre dijo que estaba encarcelado el día que mataron a Fernández Gutiérrez, y una sentencia judicial le dio la razón e

In [19]:
from datetime import date


date = date.today().strftime("%d-%m-%Y")


filename = 'Maldita_FN_' + date
print(filename)

outfile = open(filename,'wb')

pickle.dump(lista_articulos, outfile)
outfile.close()

Maldita_FN_28-05-2020


In [20]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': '¿Qué sabemos sobre la vinculación del padre de Pablo Iglesias con el FRAP?',
  'Publication_date': '28/05/2020',
  'Content': 'La portavoz del Partido Popular en el Congreso de los Diputados, Cayetana Álvarez de Toledo, ha asegurado en el Pleno de este miércoles, 27 de mayo, que el vicepresidente segundo del Gobierno, Pablo Iglesias, era “hijo de un terrorista”. Se trata de una acusación que lleva años circulando y que también fue difundida por el actual eurodiputado de Vox Hermann Tertsch, quien fue más allá y aseguró en 2018 que Francisco Javier Iglesias fue detenido por el asesinato del subinspector de Policía José Antonio Fernández Gutiérrez, ocurrido en 1973, y que fue atribuido al Frente Revolucionario Antifascista y Patriota (FRAP).Pablo Iglesias reconoció ser “hijo de un militante del FRAP” en una columna publicada en Público en el año 2012, pero su padre dijo que estaba encarcelado el día que mataron a Fernández Gutiérrez, y una sentencia judicial le dio la razón e